Для начала приведем csv файл в корекнтный формат

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split


input_file = "/content/drive/MyDrive/final_train.csv"
df = pd.read_csv(input_file)

In [4]:
texts = df['text'].values
for i in range(len(texts)):
    texts[i] = "clustering: " + texts[i]

emojis_data = df['emoji'].factorize()
emojis = emojis_data[0]
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, emojis, test_size=0.1, random_state=42)

In [5]:
emojis_data[0][:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 0])

In [6]:
map = {}
for i in range(len(emojis_data[0])):
    em = emojis_data[0][i]
    if em not in map:
        map[em] = 1
    else:
        map[em] += 1

print(len(map))

62


In [7]:
def emoji_to_num(emoji):
  for index, elem in enumerate(emojis_data[1]):
    if elem == emoji:
      return index

def num_to_emoji(num):
  return emojis_data[1][num]

Создаем пайторч датасет

Создаем модель

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        probs = F.softmax(inputs, dim=-1)
        targets_one_hot = F.one_hot(targets, num_classes=inputs.shape[-1])
        probs = torch.sum(probs * targets_one_hot, dim=-1)

        focal_weight = (1 - probs) ** self.gamma

        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        loss = focal_weight * ce_loss

        if self.alpha is not None:
            alpha_weight = self.alpha.gather(0, targets)
            loss = alpha_weight * loss

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss


find alphas

In [9]:
num_classes = 62
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

alpha = torch.zeros(num_classes).to(device)
for i in range(len(alpha)):
  alpha[i] = 8000 / map[i]


alpha2 = torch.ones(num_classes).to(device)

alpha3 = torch.sqrt(alpha)

alpha4 = torch.pow(alpha, 3/4)

In [10]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 22.1 MB/s eta 0:00:00


In [11]:
import torch.nn as nn
from transformers import BertModel
from transformers import AutoModel
from peft import LoraConfig, get_peft_model
import torch.nn.functional as F

class EmojiClassifier2(nn.Module):
    def __init__(self, num_labels, alpha):
        super(EmojiClassifier2, self).__init__()
        self.bert = AutoModel.from_pretrained("ai-forever/ru-en-RoSBERTa")

        lora_config = LoraConfig(
            r=32,
            lora_alpha=32,
            target_modules=["query", "value", "key"],
            bias="none"
        )

        self.bert = get_peft_model(self.bert, lora_config)
        self.alpha = alpha

        for param in self.bert.parameters():
            param.requires_grad = False
        for name, param in self.bert.named_parameters():
            if 'lora' in name:
                param.requires_grad = True


        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_labels)
        )

    def forward(self, input_ids, attention_mask, labels=None): # тут посмотреть
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.pooler_output
        logits = self.classifier(cls_output)

        if labels is not None:
            loss_fct = FocalLoss(gamma=1.0, alpha=alpha3)
            loss = loss_fct(logits, labels)
            return loss, logits
        return logits

In [12]:
from torch.utils.data import DataLoader
from transformers import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import LambdaLR


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = EmojiClassifier2(num_labels=len(df['emoji'].unique()), alpha=alpha3).to(device)

model_load_path = '/content/drive/MyDrive/model32lora2024-10-05_14:16:56.pth'
model.load_state_dict(torch.load(model_load_path, map_location=device))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-12-67c6516cd664>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommen

<All keys matched successfully>

In [13]:
model.eval()

with torch.no_grad():
    last_linear_layer = model.classifier[-1]
    label_embeddings = last_linear_layer.weight.detach()  # [num_labels, 512]

label_embeddings.shape

torch.Size([62, 512])

In [28]:
!pip install plotly

In [35]:
import numpy as np
import plotly.graph_objects as go
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

label_embeddings_cpu = label_embeddings.cpu()

tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(label_embeddings_cpu)

num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(label_embeddings_cpu)

fig = go.Figure()

for i in range(num_labels):
    emoji = num_to_emoji(i)
    fig.add_trace(go.Scatter(
        x=[embeddings_2d[i, 0]],
        y=[embeddings_2d[i, 1]],
        text=[emoji],
        mode='text',
        textfont=dict(size=14),
        hoverinfo='text',
        marker=dict(color=clusters[i], showscale=True)
    ))

fig.update_layout(
    title="t-SNE visualization of label embeddings with KMeans clustering",
    xaxis_title="t-SNE component 1",
    yaxis_title="t-SNE component 2",
    showlegend=False
)

fig.show()

In [62]:

num_clusters = 15
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(label_embeddings_cpu)

list_index_cluster = []
for i in range(num_clusters):
  list_index_cluster.append([])
for index, element in enumerate(clusters):
  list_index_cluster[element].append(index)


In [64]:
for i in range(num_clusters):
  res = [num_to_emoji(j) for j in list_index_cluster[i]]
  if len(res) < 7 and len(res) > 1:
    print(res)

['😒', '😔', '😢']
['🙌', '👍', '😌', '👌', '💪']
['😋', '🤤']
['😳', '🤦', '🙄', '🤡', '😱', '😡']
['❤', '🙏', '\U0001faf6', '🤍', '💋']
['🥺', '🥲', '\U0001f979', '😭']
['🤔', '🧐']
['😇', '🙃', '👀', '🌚']
['🎉', '🥳']
['😂', '😁', '😆', '🤣', '🤪']


In [95]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ru-en-RoSBERTa")

text = ["Здравствуйте, как ваши дела?",
        "Послушай песню которую я сам сочинил",
        "Знания Сила",
        "по выходным сплю дома целый день",
        "Сегодня сдал экзамен",
        "Сегодня завалил зачет",
        "Я забыл сегодня поздравить Никиту",
        "У меня очень хорошее настроение",
        "Давай выпьем пива вечером",
        "Огненное настроение",
        "Ты крайне мила",
        "Сегодня планирую работать всю ночь"
]

inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

model.eval()

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs

probabilities = torch.softmax(logits, dim=-1)

predicted_class = torch.argmax(probabilities, dim=-1)
for index in range(len(text)):
  print(f"Text: {text[index]} #### Predicted class: {num_to_emoji(predicted_class[index].item())}")

Text: Здравствуйте, как ваши дела? #### Predicted class: 😊
Text: Послушай песню которую я сам сочинил #### Predicted class: 😎
Text: Знания Сила #### Predicted class: 💪
Text: по выходным сплю дома целый день #### Predicted class: 🥲
Text: Сегодня сдал экзамен #### Predicted class: 😎
Text: Сегодня завалил зачет #### Predicted class: 😭
Text: Я забыл сегодня поздравить Никиту #### Predicted class: 😭
Text: У меня очень хорошее настроение #### Predicted class: 🥰
Text: Давай выпьем пива вечером #### Predicted class: 😉
Text: Огненное настроение #### Predicted class: 🔥
Text: Ты крайне мила #### Predicted class: ❤
Text: Сегодня планирую работать всю ночь #### Predicted class: 🥲
